# Personal Productivity OS – Google SDK Implementation

This section contains **Python code** using the **Google API SDK** (via `google-api-python-client` and `google-auth`)  
to wire up the Email and Calendar agents for the Personal Productivity OS.

> ⚠️ You will need to add your own Google Cloud project, OAuth credentials, and enable the Gmail & Calendar APIs.


## 1. Environment Setup
Install dependencies (only needed once in your environment):

In [ ]:

# Uncomment and run this cell in your own environment (not needed if already installed)
# !pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib


## 2. Google Auth Helper
Reusable helper to create authenticated service clients for Gmail and Calendar.

In [ ]:

import os
from typing import Tuple

from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

# If modifying these scopes, delete the token.json file.
GMAIL_SCOPES = ["https://www.googleapis.com/auth/gmail.modify"]
CAL_SCOPES = ["https://www.googleapis.com/auth/calendar"]


def get_credentials(scopes, credentials_file: str = "credentials.json", token_file: str = "token.json") -> Credentials:
    """Load or create OAuth2 credentials for Google APIs.

    - `credentials.json` is the client secret file downloaded from Google Cloud Console.
    - `token.json` will be created automatically after the first OAuth flow.
    """
    creds = None
    if os.path.exists(token_file):
        creds = Credentials.from_authorized_user_file(token_file, scopes)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(credentials_file, scopes)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open(token_file, "w") as token:
            token.write(creds.to_json())
    return creds


def get_gmail_service() -> any:
    creds = get_credentials(GMAIL_SCOPES, credentials_file="credentials_gmail.json", token_file="token_gmail.json")
    service = build("gmail", "v1", credentials=creds)
    return service


def get_calendar_service() -> any:
    creds = get_credentials(CAL_SCOPES, credentials_file="credentials_calendar.json", token_file="token_calendar.json")
    service = build("calendar", "v3", credentials=creds)
    return service


## 3. Base Agent Interfaces
Simple base classes for agents and a light-weight context object.

In [ ]:

from dataclasses import dataclass, field
from typing import List, Dict, Any, Optional, Protocol


@dataclass
class UserProfile:
    id: str
    email: str
    name: Optional[str] = None
    preferences: Dict[str, Any] = field(default_factory=dict)


@dataclass
class AgentInput:
    id: str
    user_id: str
    text: str
    metadata: Dict[str, Any] = field(default_factory=dict)


@dataclass
class AgentResult:
    messages: List[str] = field(default_factory=list)
    tool_calls: List[Dict[str, Any]] = field(default_factory=list)
    requires_confirmation: bool = False
    confirmation_prompt: Optional[str] = None


@dataclass
class AgentContext:
    user: UserProfile
    # In a real system, add memories, tools, logger, etc.


class Agent(Protocol):
    name: str

    def handle(self, agent_input: AgentInput, ctx: AgentContext) -> AgentResult:
        ...


## 4. Gmail Agent using Google SDK
This agent can:
- List recent emails
- Summarize subjects
- Draft and send replies (send is separated for safety).

In [ ]:

    from base64 import urlsafe_b64encode
    from email.message import EmailMessage
    from datetime import datetime


    class GmailAgent:
        name = "email"

        def __init__(self, gmail_service=None):
            self.gmail = gmail_service or get_gmail_service()

        def list_recent_messages(self, user_id: str = "me", max_results: int = 10):
            """List recent messages from the Gmail inbox."""
            results = self.gmail.users().messages().list(
                userId=user_id,
                maxResults=max_results,
                labelIds=["INBOX"]
            ).execute()
            return results.get("messages", [])

        def get_message_snippet(self, msg_id: str, user_id: str = "me") -> Dict[str, Any]:
            msg = self.gmail.users().messages().get(
                userId=user_id,
                id=msg_id,
                format="metadata",
                metadataHeaders=["Subject", "From", "Date"],
            ).execute()
            headers = {h["name"]: h["value"] for h in msg.get("payload", {}).get("headers", [])}
            return {
                "id": msg_id,
                "snippet": msg.get("snippet"),
                "subject": headers.get("Subject"),
                "from": headers.get("From"),
                "date": headers.get("Date"),
            }

        def draft_reply(
            self,
            original: Dict[str, Any],
            reply_text: str,
            user_email: str,
            user_id: str = "me",
        ) -> Dict[str, Any]:
            """Create a draft reply (does not send yet)."""
            msg = EmailMessage()
            msg["To"] = original["from"]
            msg["From"] = user_email
            msg["Subject"] = f"Re: {original['subject']}"
            msg.set_content(reply_text)

            encoded_message = urlsafe_b64encode(msg.as_bytes()).decode()
            body = {"raw": encoded_message, "threadId": original["id"]}

            draft = self.gmail.users().drafts().create(
                userId=user_id,
                body={"message": body},
            ).execute()
            return draft

        def send_message(self, raw_message: Dict[str, Any], user_id: str = "me") -> Dict[str, Any]:
            """Send a prepared raw message or draft message body."""
            sent = self.gmail.users().messages().send(userId=user_id, body=raw_message).execute()
            return sent

        def handle(self, agent_input: AgentInput, ctx: AgentContext) -> AgentResult:
            """Very simple router for email-related intents.

"
            "Examples:
"
            "- "summarize my last 5 emails"
"
            "- "show me subjects of last 10 emails"
"
            """"

            text = agent_input.text.lower()
            messages_out: List[str] = []

            if "summarize" in text or "last" in text:
                max_results = 5
                for token in text.split():
                    if token.isdigit():
                        max_results = int(token)

                msg_ids = self.list_recent_messages(max_results=max_results)
                details = [self.get_message_snippet(m["id"]) for m in msg_ids]

                summary_lines = []
                for d in details:
                    summary_lines.append(f"- {d['from']} — {d['subject']}")

                messages_out.append("Here are your most recent emails:\n" + "\n".join(summary_lines))
                return AgentResult(messages=messages_out)

            messages_out.append("I can summarize recent emails or draft replies. Try: 'summarize my last 5 emails'.")
            return AgentResult(messages=messages_out)


## 5. Calendar Agent using Google Calendar SDK
This agent can:
- Create events
- List upcoming events

In [ ]:

from datetime import datetime, timedelta, timezone


class CalendarAgent:
    name = "calendar"

    def __init__(self, calendar_service=None):
        self.calendar = calendar_service or get_calendar_service()

    def create_event(
        self,
        summary: str,
        start: datetime,
        end: datetime,
        calendar_id: str = "primary",
        timezone_str: str = "UTC",
    ) -> Dict[str, Any]:
        event_body = {
            "summary": summary,
            "start": {
                "dateTime": start.isoformat(),
                "timeZone": timezone_str,
            },
            "end": {
                "dateTime": end.isoformat(),
                "timeZone": timezone_str,
            },
        }
        event = self.calendar.events().insert(calendarId=calendar_id, body=event_body).execute()
        return event

    def list_upcoming(
        self,
        calendar_id: str = "primary",
        max_results: int = 10,
    ) -> Dict[str, Any]:
        now = datetime.now(timezone.utc).isoformat()
        events_result = self.calendar.events().list(
            calendarId=calendar_id,
            timeMin=now,
            maxResults=max_results,
            singleEvents=True,
            orderBy="startTime",
        ).execute()
        return events_result.get("items", [])

    def handle(self, agent_input: AgentInput, ctx: AgentContext) -> AgentResult:
        text = agent_input.text.lower()
        messages_out: List[str] = []

        if "focus block" in text or "focus" in text:
            # Simple example: schedule a 2-hour focus block tomorrow 9–11am (local-ish)
            tomorrow = datetime.now() + timedelta(days=1)
            start = tomorrow.replace(hour=9, minute=0, second=0, microsecond=0)
            end = start + timedelta(hours=2)

            event = self.create_event(
                summary="Deep work focus block",
                start=start,
                end=end,
                timezone_str="UTC",  # adapt to user preference in real system
            )
            messages_out.append(f"Scheduled a focus block: {event.get('htmlLink', '(event created)')}")
            return AgentResult(messages=messages_out)

        messages_out.append("I can schedule simple events. Try: 'schedule a focus block for me'.")
        return AgentResult(messages=messages_out)


## 6. Minimal Orchestrator Wiring Gmail + Calendar
This is a minimal example of the **Orchestrator Agent** that routes to Gmail or Calendar based on the user's text.

In [ ]:

class Orchestrator:
    def __init__(self, email_agent: GmailAgent, calendar_agent: CalendarAgent):
        self.email_agent = email_agent
        self.calendar_agent = calendar_agent

    def route(self, agent_input: AgentInput, ctx: AgentContext) -> AgentResult:
        text = agent_input.text.lower()

        if any(k in text for k in ["email", "inbox", "gmail"]):
            return self.email_agent.handle(agent_input, ctx)

        if any(k in text for k in ["calendar", "schedule", "meeting", "focus block", "focus"]):
            return self.calendar_agent.handle(agent_input, ctx)

        # Fallback
        return AgentResult(
            messages=[
                "I wasn't sure whether this is an email or calendar request. "
                "Try including words like 'email' or 'calendar' in your request."
            ]
        )


# --- Example usage (will only work once you configure your Google credentials) ---
if __name__ == "__main__":
    user = UserProfile(id="demo-user", email="you@example.com", name="Demo User")
    ctx = AgentContext(user=user)

    gmail_agent = GmailAgent()
    calendar_agent = CalendarAgent()
    orchestrator = Orchestrator(gmail_agent, calendar_agent)

    # Example: summarize emails
    ai = AgentInput(id="1", user_id=user.id, text="summarize my last 5 emails in my inbox")
    result = orchestrator.route(ai, ctx)
    for msg in result.messages:
        print(msg)

    # Example: schedule focus block
    ai2 = AgentInput(id="2", user_id=user.id, text="add a focus block on my calendar")
    result2 = orchestrator.route(ai2, ctx)
    for msg in result2.messages:
        print(msg)


# Using Google AI Studio (Gemini API) as the OS Brain

This section wires the **Google AI Studio / Gemini API** (via the `google-genai` SDK)  
into the Personal Productivity OS for:

- Intent classification (routing to agents)
- Task planning / decomposition
- Summaries & writing

You’ll use your **Google AI Studio API key** (Gemini Developer API).

## 1. Install and configure `google-genai`

In [ ]:

# Run this once in your environment (commented here for safety)
# !pip install -U google-genai

import os
from google import genai
from google.genai import types

# Option 1 (recommended): set GEMINI_API_KEY in your env before starting Python:
#   export GEMINI_API_KEY="your-key-here"
#
# Option 2: hard-code here (not recommended for real projects)
api_key = os.environ.get("GEMINI_API_KEY", "YOUR_GEMINI_API_KEY")

client = genai.Client(api_key=api_key)

# Quick smoke test helper (optional)
def gemini_ping():
    resp = client.models.generate_content(
        model="gemini-2.5-flash",
        contents="Reply with: OK"
    )
    return resp.text

# print(gemini_ping())


## 2. Helper: structured LLM call for agents

In [ ]:

import json
from typing import List, Dict, Any

def gemini_structured(
    system_prompt: str,
    user_text: str,
    model: str = "gemini-2.5-flash",
    response_format: str = "json",
) -> Dict[str, Any]:
    """Call Gemini with a system + user prompt and parse JSON output.

    - `system_prompt`: high-level instructions and schema.
    - `user_text`: natural-language input (goal, email, etc.).
    - `response_format`: currently only 'json' is used.
    """
    prompt = f"""{system_prompt}

USER_INPUT:
{user_text}

Respond ONLY with valid {response_format}."
"""  # noqa: E501

    resp = client.models.generate_content(
        model=model,
        contents=prompt,
    )
    raw = resp.text or ""
    try:
        return json.loads(raw)
    except json.JSONDecodeError:
        # Fallback: try to extract JSON between ``` or first/last brace
        stripped = raw.strip()
        # naive brace-based extraction
        start = stripped.find("{")
        end = stripped.rfind("}")
        if start != -1 and end != -1 and end > start:
            try:
                return json.loads(stripped[start : end + 1])
            except Exception:
                pass
        raise ValueError(f"Gemini response was not valid JSON:\n{raw}")


## 3. LLM-powered intent classifier for the Orchestrator

In [ ]:

from enum import Enum

class IntentType(str, Enum):
    PLAN_TASKS = "plan_tasks"
    MANAGE_EMAIL = "manage_email"
    MANAGE_CALENDAR = "manage_calendar"
    RESEARCH = "research"
    WRITE = "write"
    MIXED = "mixed"
    UNKNOWN = "unknown"


def classify_intent_with_gemini(user_text: str) -> Dict[str, Any]:
    system_prompt = """You are the intent router for a Personal Productivity OS.
Read the user's request and classify it into one of these intents:
  - plan_tasks
  - manage_email
  - manage_calendar
  - research
  - write
  - mixed      (if it clearly involves more than one of the above)
  - unknown

Return JSON in this exact schema:
{
  "intent": "plan_tasks | manage_email | manage_calendar | research | write | mixed | unknown",
  "reason": "short natural language explanation",
  "sub_intents": [
    {
      "intent": "plan_tasks | manage_email | manage_calendar | research | write",
      "text": "the part of the user request that corresponds to this sub-intent"
    }
  ]
}

sub_intents can be an empty list if not needed.
"""  # noqa: E501

    return gemini_structured(system_prompt, user_text)


# Quick demo (safe to run once you’ve set your API key)
if __name__ == "__main__":
    example = "Plan my week and schedule 2 focus blocks on my calendar."
    out = classify_intent_with_gemini(example)
    print(out)


## 4. Task Planning Agent powered by Gemini

In [ ]:

from dataclasses import dataclass, field
from typing import Optional

@dataclass
class Task:
    title: str
    description: Optional[str] = None
    due: Optional[str] = None   # ISO date string or natural text
    priority: Optional[int] = None  # 1 = highest
    source: str = "agent"


@dataclass
class TaskPlan:
    goal: str
    tasks: List[Task] = field(default_factory=list)
    summary: str = ""


def plan_tasks_with_gemini(goal_text: str) -> TaskPlan:
    system_prompt = """You are a task planning agent for a busy knowledge worker.
Break the user's goal or request into a small number of concrete, actionable tasks.
Prefer fewer, high-impact tasks over many tiny tasks.

Return JSON with this schema:
{
  "summary": "short human summary of the plan",
  "tasks": [
    {
      "title": "short task title",
      "description": "2-3 bullet points max, or null",
      "due": "optional natural language or ISO date, or null",
      "priority": 1
    }
  ]
}

Priority: 1 = highest, 3 = low. Use 1–3 only.
Do NOT include any other top-level keys.
"""  # noqa: E501

    data = gemini_structured(system_prompt, goal_text)
    tasks: List[Task] = []
    for t in data.get("tasks", []):
        tasks.append(
            Task(
                title=t.get("title", "").strip(),
                description=t.get("description"),
                due=t.get("due"),
                priority=t.get("priority"),
            )
        )
    return TaskPlan(goal=goal_text, tasks=tasks, summary=data.get("summary", ""))


# Example usage
if __name__ == "__main__":
    goal = "Prepare for my product strategy review next Friday."
    plan = plan_tasks_with_gemini(goal)
    print("SUMMARY:", plan.summary)
    for i, t in enumerate(plan.tasks, start=1):
        print(f"{i}. {t.title} (priority {t.priority}) - due: {t.due}")


## 5. Orchestrator wired to Gemini + agents

In [ ]:

# This orchestrator reuses the earlier AgentInput/AgentResult/AgentContext
# and routes using the Gemini-based classifier.

class GeminiOrchestrator:
    def __init__(self, agents: Dict[str, Any]):
        """agents: dict mapping agent name to instance (e.g. 'email' -> GmailAgent)."""
        self.agents = agents

    def handle(self, agent_input: AgentInput, ctx: AgentContext) -> AgentResult:
        classification = classify_intent_with_gemini(agent_input.text)
        intent = classification.get("intent", "unknown")

        # Map intent → agent name
        if intent == IntentType.PLAN_TASKS.value:
            # Use LLM-only planner for now
            plan = plan_tasks_with_gemini(agent_input.text)
            lines = ["Here's the plan:", "", plan.summary, ""]
            for idx, t in enumerate(plan.tasks, start=1):
                lines.append(f"{idx}. {t.title} (priority {t.priority}, due: {t.due})")
            return AgentResult(messages=["\n".join(lines)])

        if intent == IntentType.MANAGE_EMAIL.value and "email" in self.agents:
            return self.agents["email"].handle(agent_input, ctx)

        if intent == IntentType.MANAGE_CALENDAR.value and "calendar" in self.agents:
            return self.agents["calendar"].handle(agent_input, ctx)

        if intent == IntentType.MIXED.value:
            # Very simple mixed handling: run sub-intents in sequence
            messages_out: List[str] = []
            for sub in classification.get("sub_intents", []):
                sub_intent = sub.get("intent")
                sub_text = sub.get("text") or agent_input.text
                sub_input = AgentInput(
                    id=f"{agent_input.id}:{sub_intent}",
                    user_id=agent_input.user_id,
                    text=sub_text,
                )
                if sub_intent == IntentType.PLAN_TASKS.value:
                    plan = plan_tasks_with_gemini(sub_text)
                    messages_out.append("Task plan:\n" + plan.summary)
                elif sub_intent == IntentType.MANAGE_CALENDAR.value and "calendar" in self.agents:
                    res = self.agents["calendar"].handle(sub_input, ctx)
                    messages_out.extend(res.messages)
                elif sub_intent == IntentType.MANAGE_EMAIL.value and "email" in self.agents:
                    res = self.agents["email"].handle(sub_input, ctx)
                    messages_out.extend(res.messages)

            if not messages_out:
                messages_out.append("I detected a mixed request but couldn't route it to any available agents.")
            return AgentResult(messages=messages_out)

        # Fallback: echo classification + ask user to rephrase.
        return AgentResult(
            messages=[
                f"I classified your request as '{intent}' but don't yet know how to handle it end-to-end.",
                f"Reason: {classification.get('reason', '')}",
                "You can try: 'help me plan tasks for ...' or 'manage my calendar by ...' etc.",
            ]
        )


# Example wiring with existing GmailAgent + CalendarAgent (if defined above)
if __name__ == "__main__":
    user = UserProfile(id="demo-user", email="you@example.com", name="Demo User")
    ctx = AgentContext(user=user)

    agents: Dict[str, Any] = {}
    try:
        agents["email"] = GmailAgent()
    except Exception:
        pass

    try:
        agents["calendar"] = CalendarAgent()
    except Exception:
        pass

    orch = GeminiOrchestrator(agents=agents)

    ai = AgentInput(id="1", user_id=user.id, text="Plan my week and schedule a focus block tomorrow.")
    res = orch.handle(ai, ctx)
    for m in res.messages:
        print(m)


# Advanced Agentic AI with Google SDK + Google AI Studio (Gemini)

This section adds **intensive AI/ML agent logic** using:

- Google **AI Studio / Gemini API** (`google-genai`)
- Google **Gmail & Calendar SDKs**
- Function-calling tools to let Gemini behave like an **agent** that can call Python functions


## 1. Shared imports & Gemini client (recap)

In [ ]:

import os
from typing import List, Dict, Any, Optional

from google import genai
from google.genai import types

# Use your Google AI Studio API key
GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY", "YOUR_GEMINI_API_KEY")
client = genai.Client(api_key=GEMINI_API_KEY)


## 2. Define a `schedule_meeting` tool for Gemini function calling

In [ ]:

# Advanced tool: schedule_meeting() for Calendar integration using Gemini function calling

schedule_meeting_fn = types.FunctionDeclaration(
    name="schedule_meeting",
    description="Schedule a meeting on the user's Google Calendar.",
    parameters_json_schema={
        "type": "object",
        "properties": {
            "title": {
                "type": "string",
                "description": "Short summary of the meeting."
            },
            "date": {
                "type": "string",
                "description": "Date of the meeting (natural language or ISO date)."
            },
            "start_time": {
                "type": "string",
                "description": "Start time (e.g., '15:00', '3pm')."
            },
            "duration_minutes": {
                "type": "integer",
                "description": "Duration in minutes.",
                "default": 60,
            },
            "timezone": {
                "type": "string",
                "description": "IANA timezone string, e.g. 'Asia/Kolkata'.",
                "default": "Asia/Kolkata",
            },
            "attendees": {
                "type": "array",
                "items": {"type": "string"},
                "description": "Email addresses of attendees.",
                "default": [],
            },
        },
        "required": ["title", "date", "start_time"],
    },
)

schedule_tool = types.Tool(function_declarations=[schedule_meeting_fn])


## 3. Bridge Gemini tool → `CalendarAgent` implementation

In [ ]:

from datetime import datetime, timedelta
import dateutil.parser

def schedule_meeting_tool(
    calendar_agent,
    title: str,
    date: str,
    start_time: str,
    duration_minutes: int = 60,
    timezone_str: str = "Asia/Kolkata",
    attendees: Optional[List[str]] = None,
) -> Dict[str, Any]:
    """Implementation of the schedule_meeting tool.

    Uses the existing CalendarAgent.create_event() method.
    """
    if attendees is None:
        attendees = []

    # Very simple parse: combine date & time and let dateutil handle it
    dt_str = f"{date} {start_time}"
    start = dateutil.parser.parse(dt_str)
    end = start + timedelta(minutes=duration_minutes)

    event = calendar_agent.create_event(
        summary=title,
        start=start,
        end=end,
        timezone_str=timezone_str,
    )
    # Calendar API expects attendees inside event creation; for brevity,
    # we just include them in the returned payload or you can extend create_event().
    return {
        "eventId": event.get("id"),
        "htmlLink": event.get("htmlLink"),
        "title": title,
        "start": start.isoformat(),
        "end": end.isoformat(),
        "attendees": attendees,
    }


## 4. Agent loop: Gemini decides when to call `schedule_meeting`

In [ ]:

def run_calendar_agentic_loop(
    user_prompt: str,
    calendar_agent,
    model: str = "gemini-2.5-flash",
) -> str:
    """Agent-style loop: send prompt + tool, let Gemini propose a function call,
    execute it, then send result back for a natural language confirmation.
    """
    # 1) Ask Gemini with tool declaration
    response = client.models.generate_content(
        model=model,
        contents=[types.Content(
            role="user",
            parts=[types.Part.from_text(user_prompt)],
        )],
        config=types.GenerateContentConfig(
            tools=[schedule_tool],
        ),
    )

    # 2) Check if model decided to call our function
    if not response.function_calls:
        # No function call; just return the model's text
        return response.text or "(no response)"

    fc = response.function_calls[0]
    fn_name = fc.name
    args = dict(fc.args)

    if fn_name != "schedule_meeting":
        return f"Model requested unknown function: {fn_name}"

    # 3) Execute the function in Python (bridge to Google Calendar)
    res = schedule_meeting_tool(calendar_agent=calendar_agent, **args)

    # 4) Send function result back to Gemini to produce a user-friendly message
    function_response_part = types.Part.from_function_response(
        name="schedule_meeting",
        response=res,
    )

    final = client.models.generate_content(
        model=model,
        contents=[
            types.Content(
                role="user",
                parts=[types.Part.from_text(user_prompt)],
            ),
            # include the original function call content (for thoughts etc.)
            response.candidates[0].content,
            types.Content(
                role="user",
                parts=[function_response_part],
            ),
        ],
        config=types.GenerateContentConfig(
            tools=[schedule_tool],
        ),
    )

    return final.text or "Meeting scheduled, but model returned no text."


# Example (requires valid CalendarAgent + Google credentials)
if __name__ == "__main__":
    try:
        cal_agent = CalendarAgent()
        prompt = "Schedule a 45 minute sync with alice@example.com tomorrow at 3pm called 'Project Alpha sync'."
        print(run_calendar_agentic_loop(prompt, calendar_agent=cal_agent))
    except Exception as e:
        print("Calendar agentic demo failed (likely no credentials configured):", e)


## 5. Data / ML Analyst Agent using Gemini

In [ ]:

import pandas as pd

class DataAnalystAgent:
    """AI/ML-style analyst that:
    - Looks at a pandas DataFrame schema
    - Uses Gemini to suggest EDA steps, model types, and feature ideas
    """
    def __init__(self, model: str = "gemini-2.5-flash"):
        self.model = model

    def suggest_analysis(self, df: pd.DataFrame, goal: str) -> str:
        schema_desc = []
        for col, dtype in df.dtypes.items():
            sample_vals = df[col].dropna().astype(str).head(3).tolist()
            schema_desc.append(
                f"- {col} ({dtype}) e.g. {sample_vals}"
            )
        schema_text = "\n".join(schema_desc)

        prompt = f"""You are a senior ML engineer.
You are given a pandas DataFrame with the following columns and dtypes:
{schema_text}

The user's goal is:
{goal}

1) Suggest a short plan for EDA (3–7 bullet points).
2) Suggest 1–3 model families that would work well (e.g. tree-based, linear, neural).
3) Suggest 3–5 useful engineered features.
Be concise and actionable. Use Markdown bullets.
"""  # noqa: E501

        resp = client.models.generate_content(
            model=self.model,
            contents=prompt,
        )
        return resp.text or ""


# Demo with synthetic data
if __name__ == "__main__":
    df_demo = pd.DataFrame({
        "age": [25, 32, 45, 29, 51],
        "income": [50000, 72000, 120000, 68000, 150000],
        "city": ["Delhi", "Mumbai", "Delhi", "Bangalore", "Chennai"],
        "purchased": [0, 1, 1, 0, 1],
    })
    analyst = DataAnalystAgent()
    plan_text = analyst.suggest_analysis(df_demo, goal="predict whether a user will purchase.")
    print(plan_text)
